# Managing population consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [3]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.objects import time

c = cmdb_graph.CosmosdbClient()



In [4]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [5]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-05-23T16:08:51.626889+00:00 UTU:15989 >

In [6]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 15989}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

## Population Resource Consumption

In [12]:
from app.functions import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [15]:
objid = '7851126304352'

In [17]:
validate_resources_updated_query = f"""
g.V().has('objid','{objid}').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','7851126304352').out('has').valueMap()



,name,objid,volume,max_volume,description,replenish_rate,userguid,objtype,id
0,organics,0528623152769,1159,1159,bilogical material that can be consumed by pops,10.0,8d5b667f-b225-4641-b499-73b77558ff86,resource,0528623152769
1,common minerals,1853839760845,90,90,Iron and other common material used in constru...,NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,1853839760845
2,rare minerals,7874781257076,63,63,"lithium, silver and other rare minerals used i...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,7874781257076
3,water,0025435962279,10232,10232,"H2O ready to be consumed, either frozen or in ...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,0025435962279


In [18]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [19]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'isIdle': 'false', 'name': 'Beachbouthabra Hi...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
1,"{'name': 'Tinkin Zeporquakot', 'objid': '55505...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
2,"{'name': 'Tinkin Len', 'objid': '7880677119676...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
3,"{'isIdle': 'false', 'name': 'Beachbouthabra Be...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
4,"{'isIdle': 'false', 'name': 'Beachbouthabra Sa...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
5,"{'name': 'Tinkin Raykalbrid', 'objid': '796167...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."
6,"{'name': 'Beachbouthabra Fortchartaugui', 'obj...","{'name': 'Maivil', 'class': 'terrestrial', 'ob...","{'name': 'Nis', 'objid': '9540156108482', 'con..."


In [20]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,isIdle,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,userguid,objtype,id
0,false,Beachbouthabra Hiaton,7092251861714,0.680,0.468,0.501,0.482,0.7,6987949209694,0.4915,0.4798,0.707,8d5b667f-b225-4641-b499-73b77558ff86,pop,7092251861714
1,true,Tinkin Zeporquakot,5550515400694,0.356,0.417,0.495,0.351,0.7,1384718841923,0.4230,0.4200,0.425,8d5b667f-b225-4641-b499-73b77558ff86,pop,5550515400694
2,true,Tinkin Len,7880677119676,0.450,0.510,0.549,0.355,0.7,1384718841923,0.4520,0.4810,0.446,8d5b667f-b225-4641-b499-73b77558ff86,pop,7880677119676
3,false,Beachbouthabra Benranjore,5380932415285,0.522,0.404,0.608,0.538,0.7,6987949209694,0.5730,0.4885,0.741,8d5b667f-b225-4641-b499-73b77558ff86,pop,5380932415285
4,false,Beachbouthabra Sant,6684973836024,0.723,0.502,0.458,0.522,0.7,6987949209694,0.4900,0.4960,0.701,8d5b667f-b225-4641-b499-73b77558ff86,pop,6684973836024
5,true,Tinkin Raykalbrid,7961679440441,0.550,0.495,0.599,0.362,0.7,1384718841923,0.4805,0.4878,0.616,8d5b667f-b225-4641-b499-73b77558ff86,pop,7961679440441
6,true,Beachbouthabra Fortchartaugui,7082994639519,0.505,0.275,0.720,0.561,0.7,6987949209694,0.6405,0.4578,0.506,8d5b667f-b225-4641-b499-73b77558ff86,pop,7082994639519


In [21]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,conformity,aggression,literacy,constitution,userguid,objtype,id
0,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
1,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
2,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
3,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
4,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
5,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482
6,Nis,9540156108482,organics,"organic waste,plastics",0.570,0.580,0.420,0.430,8d5b667f-b225-4641-b499-73b77558ff86,species,9540156108482


In [22]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,atmosphere,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,userguid,objtype,id
0,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
1,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
2,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
3,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
4,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
5,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352
6,Maivil,terrestrial,7851126304352,"[{Argon: 0.0}, {Carbon Dioxide: 0.0}, {Helium:...",0.777,0.802,1.481,2264456623306,Tra,true,true,true,8d5b667f-b225-4641-b499-73b77558ff86,planet,7851126304352


First getting the list of consumption for each pop.


In [23]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [24]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 15989}

In [25]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

,location_id,consumes,pop,consumption,multi
0,7851126304352,organics,7,14,False



Then we get a list of the resources available on that location. 

In [26]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['7851126304352'], 'name': ['Maivil']},
   {'volume': [1159], 'objid': ['0528623152769'], 'name': ['organics']}]}]

In [27]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,7851126304352,organics,7,14,False,1159.0,1145.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [28]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
dtype: object

In [29]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [30]:
pre_consumption

,name,objid,volume,max_volume,description,replenish_rate,userguid,objtype,id
0,organics,0528623152769,1159,1159,bilogical material that can be consumed by pops,10.0,8d5b667f-b225-4641-b499-73b77558ff86,resource,0528623152769
1,common minerals,1853839760845,90,90,Iron and other common material used in constru...,NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,1853839760845
2,rare minerals,7874781257076,63,63,"lithium, silver and other rare minerals used i...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,7874781257076
3,water,0025435962279,10232,10232,"H2O ready to be consumed, either frozen or in ...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,0025435962279


Only the resources that are consumed will diminish, and they will stop diminishing if the level goes below `0`

In [31]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,userguid,objtype,id
0,1145,organics,0528623152769,1159,bilogical material that can be consumed by pops,10.0,8d5b667f-b225-4641-b499-73b77558ff86,resource,0528623152769
1,90,common minerals,1853839760845,90,Iron and other common material used in constru...,NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,1853839760845
2,63,rare minerals,7874781257076,63,"lithium, silver and other rare minerals used i...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,7874781257076
3,10232,water,0025435962279,10232,"H2O ready to be consumed, either frozen or in ...",NaN,8d5b667f-b225-4641-b499-73b77558ff86,resource,0025435962279


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [32]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,7851126304352,organics,7,14,False,1159.0,1145.0


In [33]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [46]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOf').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out[0]

location_id    9003968629604
consumes            organics
pop                       12
consumption               24
multi                  False
available             1157.0
remaining             1133.0
Name: 0, dtype: object


{'labels': [['location'], ['pop'], ['species']],
 'objects': [{'objid': ['9003968629604'], 'name': ['Kotlaymau']},
  {'name': ['Tauraite Len'],
   'objid': ['0670547457716'],
   'health': [0.7],
   'username': ['BillmanLocal2']},
  {'name': ['Alman'], 'objid': ['3421173158043'], 'consumes': ['organics']}]}

## Looking more closely at starvation

* Grab my testing planet, with people on it
* Lower that amount of resource on that planet to 0
* run the lower health metric
* watch the people starve and die

In [47]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').valuemap()
    """)

planet = c.clean_node(c.res[0])
planet

{'name': 'Lecarrang',
 'class': 'terrestrial',
 'objid': '2423418653337',
 'radius': 0.54,
 'mass': 0.821,
 'orbitsDistance': 0.939,
 'orbitsId': '5618858323815',
 'orbitsName': 'Senpol',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'username': 'Billmanserver',
 'objtype': 'planet',
 'id': '2423418653337'}

Grabbing that planets organics as well, and setting the amount of the organics to zero

In [48]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').has('name','Organic').property('volume',-1)
    """)

In [49]:
consumption_df.loc[consumption_df['location_id']==planet['objid'],['remaining','available']] = -1
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,9003968629604,organics,12,24,False,1157.0,1133.0


In [50]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').valuemap()
    """)

organics = c.clean_node(c.res[0])
organics

{'volume': -1,
 'name': 'Organic',
 'objid': '5788906859522',
 'max_volume': 1040,
 'description': 'bilogical material that can be consumed by pops',
 'replenish_rate': 10,
 'username': 'Billmanserver',
 'objtype': 'resource',
 'id': '5788906859522'}

Now that the resources are gone, the population should start to starve. 

In [51]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

In [52]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [53]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

""


Run the following cells again and again to watch the health go down. 

In [32]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [33]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.5999999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.5999999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.5999999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.5999999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.5999999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.5999999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.5999999999999999],[Billmanserver]


In [46]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

7 pops will starve in 1425359096973


""
